In [13]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [7]:
df = pd.read_parquet("transactions_clean.parquet")
df["date"] = pd.to_datetime(df["date"])

df["client_id"] = df["client_id"].astype("int32")
df["card_id"] = df["card_id"].astype("int32")
df["merchant_id"] = df["merchant_id"].astype("int32")
df["mcc"] = df["mcc"].astype("int16")

df["amount"] = df["amount"].astype("float32")

for c in ["use_chip", "merchant_city", "merchant_state", "zip"]:
    df[c] = df[c].astype("category")

for c in [
    "has_error",
    "err_card_credential",
    "err_authentication",
    "err_financial",
    "err_system"
]:
    df[c] = df[c].astype("int8")

df["fraud"] = df["fraud"].astype("int8")

In [3]:
df

,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,fraud,has_error,err_card_credential,err_authentication,err_financial,err_system
0,2010-01-01 00:01:00,1556,2972,-77.000000,Swipe Transaction,59935,Beulah,ND,58523.0,5499,0,0,0,0,0,0
1,2010-01-01 00:02:00,561,4575,14.570000,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,0,0,0,0,0,0
2,2010-01-01 00:02:00,1129,102,80.000000,Swipe Transaction,27092,Vista,CA,92084.0,4829,0,0,0,0,0,0
3,2010-01-01 00:06:00,848,3915,46.410000,Swipe Transaction,13051,Harwood,MD,20776.0,5813,0,0,0,0,0,0
4,2010-01-01 00:07:00,1807,165,4.810000,Swipe Transaction,20519,Bronx,NY,10464.0,5942,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8851556,2019-10-31 23:54:00,672,5001,12.930000,Chip Transaction,47508,Cosby,TN,37722.0,7230,0,0,0,0,0,0
8851557,2019-10-31 23:54:00,1384,3723,67.010002,Chip Transaction,58136,Williamson,GA,30292.0,5812,0,0,0,0,0,0
8851558,2019-10-31 23:56:00,1718,2379,1.110000,Chip Transaction,86438,West Covina,CA,91792.0,5499,0,0,0,0,0,0
8851559,2019-10-31 23:56:00,1766,2066,12.800000,Online Transaction,39261,ONLINE,ONLINE,ONLINE,5815,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8851561 entries, 0 to 8851560
Data columns (total 16 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   client_id            int32         
 2   card_id              int32         
 3   amount               float32       
 4   use_chip             category      
 5   merchant_id          int32         
 6   merchant_city        category      
 7   merchant_state       category      
 8   zip                  category      
 9   mcc                  int16         
 10  fraud                int8          
 11  has_error            int8          
 12  err_card_credential  int8          
 13  err_authentication   int8          
 14  err_financial        int8          
 15  err_system           int8          
dtypes: category(4), datetime64[ns](1), float32(1), int16(1), int32(3), int8(6)
memory usage: 321.8 MB


In [11]:
df.isnull().sum()

client_id              0
card_id                0
amount                 0
use_chip               0
merchant_id            0
merchant_city          0
merchant_state         0
zip                    0
mcc                    0
fraud                  0
has_error              0
err_card_credential    0
err_authentication     0
err_financial          0
err_system             0
dow                    0
month                  0
day                    0
date_ts                0
dtype: int64

In [5]:
df["fraud"].value_counts()

fraud
0    8841548
1      10013
Name: count, dtype: int64

---

# 📊 Fraud 데이터 사전 검증 실험 요약 (Sanity Check)

## 1️⃣ 문제 상황

* 전체 거래 수: **약 885만 건**
* Fraud 비율: **약 0.11% (10,013건)**
* 👉 **극단적 불균형 데이터**

  * 그대로 학습하면 모델이 전부 `정상(0)`만 예측해도 성능이 좋아 보이는 착시 발생

---

## 2️⃣ 실험 목적

> **“이 데이터에 fraud를 구분할 수 있는 신호(signal)가 실제로 존재하는가?”**

즉,

* 운영 성능을 바로 보려는 실험 ❌
* **데이터 자체가 모델링 가치가 있는지 검증**하는 단계 ✅

---

## 3️⃣ 실험 방법 (의도적으로 단순하게 설계)

### (1) 데이터 구성

* Fraud(1): **전부 사용**
* Non-fraud(0): Fraud의 **1.2배만 랜덤 샘플링**

  * → 학습이 가능한 **준-균형 데이터셋** 구성

### (2) 반복 실험

* 위 과정을 **20번 반복**
* 매번:

  * 랜덤 undersampling
  * stratified train/test split
  * Logistic Regression 학습

👉 **샘플링 운빨 제거 + 결과 안정성 확인 목적**

---

## 4️⃣ 사용한 피처

* 모든 거래 단위 기본 정보 사용

  * 금액(amount)
  * 결제 방식(use_chip)
  * 가맹점 위치(city/state)
  * 업종(mcc)
  * 카드/고객 정보
  * 에러 플래그들
  * 시간 파생 변수(dow, month, day, timestamp)

---

## 5️⃣ 결과 요약

### 📈 반복 실험 결과 (20 runs)

* **ROC-AUC**

  * 평균: **0.865**
  * 표준편차: **0.007**
* **PR-AUC**

  * 평균: **0.834**
  * 표준편차: **0.017**

👉 run 간 편차가 매우 작음 → **우연이 아닌 구조적 신호**

---

## 6️⃣ 이 결과가 의미하는 것

### ✅ 긍정적인 점

* 단순한 Logistic Regression만으로도

  * fraud / non-fraud 구분 가능
* 데이터/라벨/피처에 **명확한 판별 신호 존재**
* 모델이 **안정적으로 학습됨**

### ❗ 주의할 점

* 이 점수는 **균형 데이터 기준**
* 실제 운영 분포(0.11%)에서의 성능을 의미하지는 않음
* 즉,

  * **“이 모델을 바로 쓸 수 있다” ❌**
  * **“이 데이터로 모델링을 진행할 가치가 있다” ✅**

---

## 7️⃣ 결론 (한 줄 요약)

> **“현재 데이터는 fraud 탐지를 위한 의미 있는 신호를 충분히 포함하고 있으며,
> 추가적인 운영 환경 평가(원래 분포 기준, Top-K/threshold 평가)를 진행할 가치가 있다.”**

---

## 8️⃣ 다음 단계 (실전용)

이제 아래 단계로 넘어가면 **실제 적용 가능성**을 판단할 수 있음:

1. 원래 분포 test에서 **Top-K precision / recall**
2. client_id 기준 **Group split**
3. Tree 계열 모델(XGBoost/LightGBM) 비교

---


In [ ]:
# 0) Data 준비

df2 = df.copy()

TARGET = "fraud"
X_cols = [c for c in df2.columns if c != TARGET]

# pos / neg 분리
pos_df = df2[df2[TARGET] == 1]
neg_df = df2[df2[TARGET] == 0]

n_pos = len(pos_df)
print("Positives:", n_pos)
print("Negatives pool:", len(neg_df))

# neg : pos = 1.2 : 1  (예전 실험과 동일)
NEG_POS_RATIO = 1.2
n_neg_target = int(np.ceil(n_pos * NEG_POS_RATIO))
n_neg_target = min(n_neg_target, len(neg_df))

print("Neg/Pos target:", n_neg_target / n_pos)

# 1) 반복 실험

N_RUNS = 20

roc_list = []
pr_list = []

# categorical / numeric
cat_cols = df2[X_cols].select_dtypes(include=["category", "object"]).columns.tolist()
num_cols = [c for c in X_cols if c not in cat_cols]

for run in range(N_RUNS):
    # ---- negative undersampling
    neg_sampled = neg_df.sample(
        n=n_neg_target,
        replace=False,
        random_state=run
    )

    df_bal = (
        pd.concat([pos_df, neg_sampled], axis=0)
        .sample(frac=1, random_state=run)
        .reset_index(drop=True)
    )

    X = df_bal[X_cols].copy()
    y = df_bal[TARGET].astype(int)

    # ---- stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        stratify=y,
        random_state=run
    )

    # ---- categorical encoding
    X_train[cat_cols] = X_train[cat_cols].astype(str)
    X_test[cat_cols]  = X_test[cat_cols].astype(str)

    enc = OrdinalEncoder(
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )

    X_train[cat_cols] = enc.fit_transform(X_train[cat_cols])
    X_test[cat_cols]  = enc.transform(X_test[cat_cols])

    # ---- model
    clf = LogisticRegression(
        max_iter=2000,
        n_jobs=-1,
        solver="lbfgs"
    )

    clf.fit(X_train, y_train)

    y_prob = clf.predict_proba(X_test)[:, 1]

    roc = roc_auc_score(y_test, y_prob)
    pr  = average_precision_score(y_test, y_prob)

    roc_list.append(roc)
    pr_list.append(pr)

    print(f"[Run {run+1:02d}] ROC={roc:.4f}, PR={pr:.4f}")

print("\n================ FINAL SUMMARY ================")
print(f"ROC-AUC : mean={np.mean(roc_list):.4f}, std={np.std(roc_list):.4f}")
print(f"PR-AUC  : mean={np.mean(pr_list):.4f}, std={np.std(pr_list):.4f}")


Positives: 10013
Negatives pool: 8841548
Neg/Pos target: 1.2000399480675121
[Run 01] ROC=0.8761, PR=0.8639
[Run 02] ROC=0.8743, PR=0.8569
[Run 03] ROC=0.8696, PR=0.8310
[Run 04] ROC=0.8638, PR=0.8207
[Run 05] ROC=0.8618, PR=0.8221
[Run 06] ROC=0.8808, PR=0.8693
[Run 07] ROC=0.8591, PR=0.8175
[Run 08] ROC=0.8521, PR=0.8155
[Run 09] ROC=0.8626, PR=0.8418
[Run 10] ROC=0.8648, PR=0.8427
[Run 11] ROC=0.8587, PR=0.8168
[Run 12] ROC=0.8594, PR=0.8343
[Run 13] ROC=0.8630, PR=0.8318
[Run 14] ROC=0.8601, PR=0.8198
[Run 15] ROC=0.8618, PR=0.8141
[Run 16] ROC=0.8652, PR=0.8228
[Run 17] ROC=0.8570, PR=0.8328
[Run 18] ROC=0.8663, PR=0.8219
[Run 19] ROC=0.8690, PR=0.8453
[Run 20] ROC=0.8770, PR=0.8558

================ FINAL SUMMARY ================
ROC-AUC : mean=0.8651, std=0.0072
PR-AUC  : mean=0.8338, std=0.0166
